In [22]:
#include json and os librareries
import json
import time
import pandas as pd
import os

In [28]:
####Globals:
JSON_DIR="ExtractTestsResults/JsonTestsFolders/cyclicts/cyclictest-"
TEST_TYPE = "cyclictest"

############################################################################
#                                                                          #
#                    #################################                     #
#                    #      Helper functions         #                     #
#                    #################################                     #
#                                                                          #
############################################################################

def create_json(results_dict:dict, JSON_DIR:str):
    #create json object
    #add uniuqe id to file name (using unix time)
    timestamp = str(time.time()).replace('.', '')
    # open a file and write the JSON data to it
    with open(JSON_DIR+timestamp+".json", "w") as json_file:
        json.dump(results_dict, json_file)

def list_files(path):
    """
    Return a list of all json file names in the specified folder.
    """
    files = []
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            files.append(file)
    return files

def extract_test_units(results_dict:dict, test_units_df):
    """
    extract all test units metrics(thread/cores) of a specific test, and update them in the result dict.\n
    params:
        -results_dict(dict)-dict that contains test parameters and metrics.\n
        -test_units_df(dataframe)- dataframe that contains specific test's test units metrics.\n
    returns: updated dict that contains test parameters and metrics.\n.
    """
    test_units_list = []
    for _, row in test_units_df.iterrows():
        test_units_list.append({"index":row["Index"], 
            "max_latency":row["Max"], 
            "min_latency":row["Min"], 
            "avg_latency":row["Avg"], 
            "availability":row["Availability"], 
            "number_of_nines":row["Number of Nines"]})

    results_dict["test_units"] = test_units_list

    return results_dict   


In [29]:
availabilty_df = pd.read_csv("spreadsheets/Informal - RAN QE Performance KPIs - cyclictest.csv").fillna("")
availabilty_df["Version"] = availabilty_df["Version"].astype("str")
availabilty_df["Operator Version"] = availabilty_df["Operator Version"].astype("str")
availabilty_df.loc[availabilty_df["Version"] == "4.1", "Version"] = "4.10"
availabilty_df.loc[availabilty_df["Operator Version"] == "4.1", "Operator Version"] = "4.10"
if not set(["Kernel", "Sideloaded"]).issubset(availabilty_df.columns):
    availabilty_df["Kernel"] = ""
    availabilty_df["Sideloaded"] = ""

In [30]:
grouped_availabilty = availabilty_df.groupby(["Version", "OCP Version"])

In [31]:
for version_key, threads in grouped_availabilty:
    print("creating json file for {}".format(version_key))
    threads = threads.sort_values(by='Index', ascending=True)
    results_dict = {}
    results_dict["ocp_version"] = threads["Version"].values[0]
    results_dict["ocp_build"] = threads["OCP Version"].values[0]
    results_dict["test_type"] = TEST_TYPE
    results_dict["node_name"] = threads["Cluster"].values[0]
    results_dict["duration"] = threads["Duration"].values[0]
    results_dict["kernel"] = threads["Kernel"].values[0]
    results_dict["sideloaded"] = "true" if threads["Sideloaded"].values[0] == "Yes" else "false" if threads["Sideloaded"].values[0] == "No" else "" 
    results_dict["operator_version"] = threads["Operator Version"].values[0]

    results_dict = extract_test_units(results_dict=results_dict, test_units_df=threads)
    create_json(results_dict=results_dict, JSON_DIR=JSON_DIR)

creating json file for ('4.10', '4.10.23')
creating json file for ('4.11', '4.11.0-rc.1')
creating json file for ('4.11', '4.11.0-rc.5')
creating json file for ('4.11', '4.11.6')
creating json file for ('4.12', '4.12.0-0.nightly-2022-10-24-103753')
creating json file for ('4.12', '4.12.0-0.nightly-2022-10-24-103753-spr')
creating json file for ('4.12', '4.12.0-0.nightly-2022-10-24-103753-telco')
creating json file for ('4.12', '4.12.0-rc.1')
creating json file for ('4.13', '4.13.0-0.ci.test-2023-03-02-214629-ci-ln-xw171tb-latest')
creating json file for ('4.13', '4.13.0-0.nightly-2023-02-16-120330')
